## Global Configuration

These settings will be applied to all backbones.

In [ ]:
import os
import sys
import json
import time
import pandas as pd
from pathlib import Path

# ========== GLOBAL CONFIGURATION ==========
ENABLE_FINETUNING = False  # Set True to enable light finetuning
USE_SOFT_ARGMAX = False    # Set True to use window soft-argmax

# Finetuning hyperparameters
FINETUNE_K_LAYERS = 2
FINETUNE_LR = 1e-5
FINETUNE_WD = 1e-4
FINETUNE_EPOCHS = 3

# Soft-argmax hyperparameters
SOFT_WINDOW = 7
SOFT_TAU = 0.05

# PCK thresholds to evaluate
PCK_ALPHAS = [0.05, 0.10, 0.15, 0.20]

print("="*60)
print("AML PROJECT ORCHESTRATOR")
print("="*60)
print(f"Configuration:")
print(f"  ENABLE_FINETUNING: {ENABLE_FINETUNING}")
print(f"  USE_SOFT_ARGMAX: {USE_SOFT_ARGMAX}")
if ENABLE_FINETUNING:
    print(f"  Finetuning: k={FINETUNE_K_LAYERS}, lr={FINETUNE_LR}, epochs={FINETUNE_EPOCHS}")
if USE_SOFT_ARGMAX:
    print(f"  Soft-argmax: window={SOFT_WINDOW}, tau={SOFT_TAU}")
print(f"  PCK thresholds: {PCK_ALPHAS}")
print("="*60)

## Setup Paths

In [ ]:
# Detect environment
if 'COLAB_GPU' in os.environ or 'google.colab' in sys.modules:
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_ROOT = '/content/drive/MyDrive/AMLProject'
else:
    IN_COLAB = False
    PROJECT_ROOT = os.path.dirname(os.path.abspath('__file__'))
    if not os.path.exists(PROJECT_ROOT):
        PROJECT_ROOT = os.getcwd()

# Define paths
DATA_ROOT = os.path.join(PROJECT_ROOT, 'data')
SPAIR_PATH = os.path.join(DATA_ROOT, 'SPair-71k')
CKPT_ROOT = os.path.join(PROJECT_ROOT, 'checkpoints')
OUTPUT_ROOT = os.path.join(PROJECT_ROOT, 'outputs')
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# Notebook paths
NOTEBOOK_DINOV2 = os.path.join(PROJECT_ROOT, 'DINOv2_Correspondence.ipynb')
NOTEBOOK_DINOV3 = os.path.join(PROJECT_ROOT, 'DINOv3_Correspondence.ipynb')
NOTEBOOK_SAM = os.path.join(PROJECT_ROOT, 'SAM_Correspondence.ipynb')

print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"DATA_ROOT: {DATA_ROOT}")
print(f"OUTPUT_ROOT: {OUTPUT_ROOT}")
print(f"\nNotebooks:")
print(f"  DINOv2: {'✓' if os.path.exists(NOTEBOOK_DINOV2) else '✗'} {NOTEBOOK_DINOV2}")
print(f"  DINOv3: {'✓' if os.path.exists(NOTEBOOK_DINOV3) else '✗'} {NOTEBOOK_DINOV3}")
print(f"  SAM: {'✓' if os.path.exists(NOTEBOOK_SAM) else '✗'} {NOTEBOOK_SAM}")

## Notebook Execution Utilities

In [ ]:
def inject_config_into_notebook(notebook_path, enable_ft, use_soft):
    """
    Inject configuration flags into a notebook by modifying the config cell.
    Returns modified notebook content as dict.
    """
    import nbformat
    
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
    
    # Find config cell (contains "ENABLE_FINETUNING = ")
    for cell in nb.cells:
        if cell.cell_type == 'code' and 'ENABLE_FINETUNING' in cell.source:
            # Replace flag values
            lines = cell.source.split('\n')
            new_lines = []
            for line in lines:
                if line.strip().startswith('ENABLE_FINETUNING'):
                    new_lines.append(f'ENABLE_FINETUNING = {enable_ft}')
                elif line.strip().startswith('USE_SOFT_ARGMAX'):
                    new_lines.append(f'USE_SOFT_ARGMAX = {use_soft}')
                else:
                    new_lines.append(line)
            cell.source = '\n'.join(new_lines)
            break
    
    return nb


def execute_notebook(notebook_path, backbone_name, enable_ft=False, use_soft=False):
    """
    Execute a notebook with specified configuration.
    Returns execution results and timing.
    """
    import nbformat
    from nbconvert.preprocessors import ExecutePreprocessor
    
    print(f"\n{'='*60}")
    print(f"Executing {backbone_name}")
    print(f"  Finetuning: {enable_ft}, Soft-argmax: {use_soft}")
    print(f"{'='*60}")
    
    t_start = time.time()
    
    try:
        # Inject config
        nb = inject_config_into_notebook(notebook_path, enable_ft, use_soft)
        
        # Execute
        ep = ExecutePreprocessor(timeout=3600, kernel_name='python3')
        ep.preprocess(nb, {'metadata': {'path': os.path.dirname(notebook_path)}})
        
        # Save executed notebook
        output_nb_path = os.path.join(
            OUTPUT_ROOT, 
            f"{backbone_name}_executed_ft{enable_ft}_soft{use_soft}.ipynb"
        )
        with open(output_nb_path, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        
        elapsed = time.time() - t_start
        
        print(f"\n✓ {backbone_name} completed in {elapsed:.1f}s")
        print(f"  Output saved to: {output_nb_path}")
        
        return {
            'backbone': backbone_name,
            'finetuning': enable_ft,
            'soft_argmax': use_soft,
            'success': True,
            'elapsed_time': elapsed,
            'output_notebook': output_nb_path
        }
        
    except Exception as e:
        elapsed = time.time() - t_start
        print(f"\n✗ {backbone_name} failed after {elapsed:.1f}s")
        print(f"  Error: {str(e)}")
        
        return {
            'backbone': backbone_name,
            'finetuning': enable_ft,
            'soft_argmax': use_soft,
            'success': False,
            'elapsed_time': elapsed,
            'error': str(e)
        }

print("✓ Notebook execution utilities loaded")

## Execute All Backbones

In [ ]:
# Execute all three backbones
results = []

backbones = [
    ('DINOv2', NOTEBOOK_DINOV2),
    ('DINOv3', NOTEBOOK_DINOV3),
    ('SAM', NOTEBOOK_SAM)
]

for backbone_name, notebook_path in backbones:
    if not os.path.exists(notebook_path):
        print(f"\n⚠️  Skipping {backbone_name}: notebook not found")
        results.append({
            'backbone': backbone_name,
            'finetuning': ENABLE_FINETUNING,
            'soft_argmax': USE_SOFT_ARGMAX,
            'success': False,
            'error': 'Notebook not found'
        })
        continue
    
    result = execute_notebook(
        notebook_path, 
        backbone_name,
        enable_ft=ENABLE_FINETUNING,
        use_soft=USE_SOFT_ARGMAX
    )
    results.append(result)

print("\n" + "="*60)
print("ALL EXECUTIONS COMPLETED")
print("="*60)

## Extract and Consolidate Results

In [ ]:
def extract_pck_from_notebook(notebook_path):
    """
    Extract PCK results from executed notebook outputs.
    Returns dict with PCK values at different thresholds.
    """
    import nbformat
    
    if not os.path.exists(notebook_path):
        return None
    
    try:
        with open(notebook_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
        
        pck_results = {}
        
        # Search for cells with PCK output
        for cell in nb.cells:
            if cell.cell_type == 'code' and cell.get('outputs'):
                for output in cell.outputs:
                    if output.output_type == 'stream':
                        text = output.get('text', '')
                        # Look for PCK@X.XX patterns
                        import re
                        matches = re.findall(r'PCK@([0-9.]+):\s*([0-9.]+)', text)
                        for alpha, value in matches:
                            pck_results[f'PCK@{alpha}'] = float(value)
        
        return pck_results if pck_results else None
        
    except Exception as e:
        print(f"Error extracting PCK from {notebook_path}: {e}")
        return None


# Extract PCK from all executed notebooks
for result in results:
    if result['success'] and 'output_notebook' in result:
        pck_data = extract_pck_from_notebook(result['output_notebook'])
        if pck_data:
            result['pck'] = pck_data
        else:
            result['pck'] = {}

print("✓ Results extracted from notebooks")

## Generate Comparison Tables

In [ ]:
# Create results DataFrame
rows = []
for result in results:
    row = {
        'Backbone': result['backbone'],
        'Finetuning': 'Yes' if result['finetuning'] else 'No',
        'Soft-Argmax': 'Yes' if result['soft_argmax'] else 'No',
        'Success': '✓' if result['success'] else '✗',
        'Time (s)': f"{result.get('elapsed_time', 0):.1f}"
    }
    
    # Add PCK values
    pck = result.get('pck', {})
    for alpha in PCK_ALPHAS:
        key = f'PCK@{alpha:.2f}'
        row[key] = f"{pck.get(key, 0.0):.4f}"
    
    rows.append(row)

df = pd.DataFrame(rows)

print("\n" + "="*80)
print("CONSOLIDATED RESULTS")
print("="*80)
print(df.to_string(index=False))
print("="*80)

# Save to CSV
csv_path = os.path.join(OUTPUT_ROOT, 'consolidated_results.csv')
df.to_csv(csv_path, index=False)
print(f"\n✓ Results saved to: {csv_path}")

# Save detailed JSON
json_path = os.path.join(OUTPUT_ROOT, 'consolidated_results.json')
with open(json_path, 'w') as f:
    json.dump(results, f, indent=2)
print(f"✓ Detailed results saved to: {json_path}")

## Best Model Selection

In [ ]:
# Find best model by PCK@0.10
best_pck = 0.0
best_model = None

for result in results:
    if result['success']:
        pck_010 = result.get('pck', {}).get('PCK@0.10', 0.0)
        if pck_010 > best_pck:
            best_pck = pck_010
            best_model = result

if best_model:
    print("\n" + "="*60)
    print("BEST MODEL")
    print("="*60)
    print(f"Backbone: {best_model['backbone']}")
    print(f"Finetuning: {'Yes' if best_model['finetuning'] else 'No'}")
    print(f"Soft-Argmax: {'Yes' if best_model['soft_argmax'] else 'No'}")
    print(f"\nPerformance:")
    for key, val in best_model.get('pck', {}).items():
        print(f"  {key}: {val:.4f}")
    print(f"\nExecution time: {best_model['elapsed_time']:.1f}s")
    print("="*60)
else:
    print("\n⚠️  No successful executions to determine best model")

## Visualization

In [ ]:
import matplotlib.pyplot as plt

# Plot PCK comparison
fig, ax = plt.subplots(figsize=(12, 6))

for result in results:
    if result['success'] and result.get('pck'):
        label = f"{result['backbone']}"
        if result['finetuning']:
            label += " (FT)"
        if result['soft_argmax']:
            label += " (Soft)"
        
        alphas = []
        pcks = []
        for alpha in PCK_ALPHAS:
            key = f'PCK@{alpha:.2f}'
            if key in result['pck']:
                alphas.append(alpha)
                pcks.append(result['pck'][key])
        
        if alphas:
            ax.plot(alphas, pcks, marker='o', label=label, linewidth=2)

ax.set_xlabel('PCK Threshold (α)', fontsize=12)
ax.set_ylabel('PCK Score', fontsize=12)
ax.set_title('PCK Performance Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 1])

plt.tight_layout()
plot_path = os.path.join(OUTPUT_ROOT, 'pck_comparison.png')
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Plot saved to: {plot_path}")

## Summary

**What to do next:**

1. **Review Results**: Check `outputs/consolidated_results.csv` for performance comparison
2. **Analyze Best Model**: See the "Best Model" section above
3. **Enable Finetuning**: Set `ENABLE_FINETUNING=True` and re-run to see improvement
4. **Enable Soft-Argmax**: Set `USE_SOFT_ARGMAX=True` to test sub-pixel refinement
5. **Ablation Study**: Try different combinations:
   - Baseline: `FT=False, Soft=False`
   - Only Soft-argmax: `FT=False, Soft=True`
   - Only Finetuning: `FT=True, Soft=False`
   - Full Pipeline: `FT=True, Soft=True`

**Expected Results:**
- Training-free baseline provides initial correspondence quality
- Soft-argmax should improve PCK@0.05 (sub-pixel accuracy)
- Light finetuning should boost all PCK metrics
- Combined approach (FT + Soft) should achieve best performance